# Keras MLP -> best Log Loss 0.555
## Overview ##
This kernel uses seeding and relative seeding with an MLP network. Spread on loss was very small, so kfold not necessary

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, TensorBoard, ProgbarLogger
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


Using TensorFlow backend.


Data
GiantKillerInfo.txt
KaggleKernelwSeeds.ipynb
KerasBinaryClassifier_Intro.ipynb
KerasMLP_only2003.csv
KerasMLP_onlyBPI_2009-2013.csv
KerasMLP_onlyseed.csv
KerasMLP_onlyseeds.csv
KerasMLP_RankAndSeeds.ipynb
KerasMLP_Rank_Kfold.ipynb
KerasMLP_RankSeedsYear_1_2.csv
KerasMLP_RankSeedsYear_1.csv
KerasMLP_RankSeedsYear.csv
KerasMLP_RanksKfold.ipynb
KerasMLP_SeedsandYear_1.csv
KerasMLP_SeedsandYear.csv
KerasMLP_SeedsOnly.ipynb
KerasMLP_SeedswithTB.ipynb
logreg_withseedtoo.csv
logs
MasseyOrdinalsExp.ipynb
MLP_first_go.csv
MLP_with_seeds.csv
sonar.csv
SVM_first_go.csv
Tensorflow.ipynb



### Lock Random Seed

In [2]:
seed = 13
np.random.seed(seed)

### Load Training data


In [3]:
data_dir = '../NCAA/Data/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


Incorporate Massey Ordinals - MAS, SAG, POM

In [4]:
#Load Data
df_rank = pd.read_csv(data_dir+ 'MasseyOrdinals.csv')
#Choose Latest Ranking 
df_rank = df_rank[df_rank.RankingDayNum>=133]
df_rank = df_rank[df_rank.Season>=2010]

#Selectr rankings of interest and make each ranking system ranking into a separate column
df_merge = pd.merge(df_rank.loc[df_rank['SystemName']=='MAS'],
                    df_rank.loc[df_rank['SystemName']=='SAG'], how='left', 
                    on=['Season', 'TeamID', 'RankingDayNum'])
df_rank = pd.merge(left=df_merge, right=df_rank.loc[df_rank['SystemName']=='POM'], 
                  how='left', on=['Season', 'TeamID', 'RankingDayNum'] )

In [5]:
# Drop
df_rank.drop(labels=['SystemName_x', 'SystemName_y', 'SystemName','RankingDayNum'], inplace=True, axis =1)
df_rank.rename(columns={'OrdinalRank_x':'MAS', 'OrdinalRank_y':'SAG', 'OrdinalRank':'POM'}, inplace=True)

df_rank.head()

,Season,TeamID,MAS,SAG,POM
0,2010,1102,239,237,246
1,2010,1103,89,105,119
2,2010,1104,79,72,64
3,2010,1105,329,337,336
4,2010,1106,276,300,302


Cut seasons prior to Max Ranking date

In [6]:
df_seeds = df_seeds[df_seeds.Season>=min(df_rank.Season) ]
df_seeds = df_seeds[ df_seeds.Season<=max(df_rank.Season)]
df_tour = df_tour[df_tour.Season>=min(df_rank.Season)]
df_tour = df_tour[ df_tour.Season<=max(df_rank.Season)]
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1584,2010,134,1115,61,1457,44,N,0
1585,2010,136,1124,68,1358,59,N,0
1586,2010,136,1139,77,1431,59,N,0
1587,2010,136,1140,99,1196,92,N,2
1588,2010,136,1242,90,1250,74,N,0


Cut off the region identifier from the seed number

In [7]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

In [8]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
# df_tour.head()

Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [9]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_seeds.head()

,Season,TeamID,seed_int
1609,2010,1246,1
1610,2010,1452,2
1611,2010,1307,3
1612,2010,1458,4
1613,2010,1396,5


In [10]:
df_dummy = pd.merge(left=df_tour, right=df_lossSeeds, how='left', on=['Season', 'LTeamID'])
df_dummy.head()

,Season,WTeamID,LTeamID,LSeed
0,2010,1115,1457,16
1,2010,1124,1358,14
2,2010,1139,1431,12
3,2010,1140,1196,10
4,2010,1242,1250,16


In [11]:
df_concat = pd.merge(left=df_dummy, right=df_winSeeds, how='left' ,on=['Season', 'WTeamID'])
# df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

,Season,WTeamID,LTeamID,LSeed,WSeed
0,2010,1115,1457,16,16
1,2010,1124,1358,14,3
2,2010,1139,1431,12,5
3,2010,1140,1196,10,7
4,2010,1242,1250,16,1


Dataframe with seed, relative seed, and win/loss. Use for training

In [12]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
# df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins = pd.merge(left=df_wins, right=df_rank, how='left', on=['Season', 'TeamID'])
df_wins.head()
df_wins.head()

,Seed,TeamID,Season,Result,MAS,SAG,POM
0,16,1115,2010,1,232,239,238
1,3,1124,2010,1,11,8,12
2,5,1139,2010,1,13,22,26
3,7,1140,2010,1,14,9,7
4,1,1242,2010,1,1,1,2


In [13]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
# df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses = pd.merge(left=df_losses, right=df_rank, how='left',on=['Season', 'TeamID'])
df_losses.head()

,Seed,TeamID,Season,Result,MAS,SAG,POM
0,16,1457,2010,0,192,210,212
1,14,1358,2010,0,96,106,102
2,12,1431,2010,0,28,33,34
3,10,1196,2010,0,55,48,49
4,16,1250,2010,0,201,204,180


In [14]:
df_lossesOpp = df_losses.copy()
df_lossesOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_lossesOpp.columns.values]
df_lossesOpp.rename(columns = dict(new_names), inplace=True)
df_lossesOpp.head()

,OppSeed,OppTeamID,OppMAS,OppSAG,OppPOM
0,16,1457,192,210,212
1,14,1358,96,106,102
2,12,1431,28,33,34
3,10,1196,55,48,49
4,16,1250,201,204,180


In [15]:
df_winsOpp = df_wins.copy()
df_winsOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_winsOpp.columns.values]
df_winsOpp.rename(columns = dict(new_names), inplace=True)
df_winsOpp.head()

,OppSeed,OppTeamID,OppMAS,OppSAG,OppPOM
0,16,1115,232,239,238
1,3,1124,11,8,12
2,5,1139,13,22,26
3,7,1140,14,9,7
4,1,1242,1,1,2


In [16]:
df_winloss = pd.concat([df_wins, df_lossesOpp], axis=1)

df_losswin = pd.concat([df_losses, df_winsOpp], axis=1)
df_losswin.head()

,Seed,TeamID,Season,Result,MAS,SAG,POM,OppSeed,OppTeamID,OppMAS,OppSAG,OppPOM
0,16,1457,2010,0,192,210,212,16,1115,232,239,238
1,14,1358,2010,0,96,106,102,3,1124,11,8,12
2,12,1431,2010,0,28,33,34,5,1139,13,22,26
3,10,1196,2010,0,55,48,49,7,1140,14,9,7
4,16,1250,2010,0,201,204,180,1,1242,1,1,2


In [17]:
df_finalData = pd.concat((df_winloss, df_losswin))
results = df_finalData['Result']
df_finalData.drop(labels=['Result'], inplace=True, axis=1)
df_finalData.insert(0, 'Result', results)
df_finalData.head()

,Result,Seed,TeamID,Season,MAS,SAG,POM,OppSeed,OppTeamID,OppMAS,OppSAG,OppPOM
0,1,16,1115,2010,232,239,238,16,1457,192,210,212
1,1,3,1124,2010,11,8,12,14,1358,96,106,102
2,1,5,1139,2010,13,22,26,12,1431,28,33,34
3,1,7,1140,2010,14,9,7,10,1196,55,48,49
4,1,1,1242,2010,1,1,2,16,1250,201,204,180


In [18]:
df_finalData.isnull().any()

Result       False
Seed         False
TeamID       False
Season       False
MAS          False
SAG          False
POM          False
OppSeed      False
OppTeamID    False
OppMAS       False
OppSAG       False
OppPOM       False
dtype: bool

 ### Set feature ouput and feature vector, split into training and test

In [19]:
X = df_finalData.iloc[:,1:]
# Set input_dim
xDim = np.shape(X)[1]
X= X.values.reshape(-1,xDim)

y = df_finalData.Result.values
trainSize = int(np.shape(X)[0]*.8)
x_train, x_test = X[:trainSize, :], X[trainSize:,:]
y_train, y_test = y[:trainSize], y[trainSize:]

print('Feature vector dimension is: %.2f' % xDim)
print(X[:5,:])

Feature vector dimension is: 11.00
[[  16 1115 2010  232  239  238   16 1457  192  210  212]
 [   3 1124 2010   11    8   12   14 1358   96  106  102]
 [   5 1139 2010   13   22   26   12 1431   28   33   34]
 [   7 1140 2010   14    9    7   10 1196   55   48   49]
 [   1 1242 2010    1    1    2   16 1250  201  204  180]]


## MLP Props
droput rate, batch size, epoch size

In [20]:
dropRate = 0.5
numBatch = 5
numEpoch = 100


###  MLP Base, 10 Nodes

In [21]:
# base model
base10 = Sequential()
base10.name = 'base10'
base10.add(Dense(10, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
base10.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
base10.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### MLP Base 20 Nodes

In [22]:
# base model
base20 = Sequential()
base20.name = 'base20'
base20.add(Dense(20, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
base20.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
base20.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Dropout Model 

In [23]:
# dropout model
dropmod = Sequential()
dropmod.name = 'dropmod'
dropmod.add(Dense(10, input_dim=xDim, kernel_initializer='normal',activation = 'relu'))
dropmod.add(Dropout(dropRate))
dropmod.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
dropmod.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Dense Model

In [24]:
# dense model
dense = Sequential()
dense.name='dense'
dense.add(Dense(10, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
dense.add(Dropout(dropRate))
dense.add(Dense(5, input_dim=xDim, kernel_initializer='normal',  activation = 'relu'))
dense.add(Dropout(dropRate))
dense.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
dense.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Denser Model with Dropout

Has not shown good results, cut 2/22

In [25]:
# dense model
densedrop = Sequential()
densedrop.name = 'densedrop'
densedrop.add(Dense(10, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
densedrop.add(Dropout(dropRate))
densedrop.add(Dense(5, input_dim=xDim, kernel_initializer='normal',  activation = 'relu'))
densedrop.add(Dropout(dropRate))
densedrop.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
densedrop.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fit the Model 

### Models of Interest

In [26]:
model = [base10, base20, dropmod, dense, densedrop ]

###  StandardScaler


In [27]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


###  with scaled data

In [28]:
for i in range(0,np.size(model)):  
    TB = TensorBoard(log_dir="logs/{}_{}{}".format(str(model[i].name), 
                                                   localtime().tm_hour, localtime().tm_min))
    model[i].fit(X, y, validation_split = 0.2, shuffle=True, epochs = numEpoch, batch_size = numBatch,
                 verbose=0, callbacks=[TB, ProgbarLogger])#, EarlyStopping( patience=5)])

TypeError: set_model() missing 1 required positional argument: 'model'

In [ ]:
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [ ]:
X_test = np.zeros(shape=(n_test_games, xDim))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    diff_seed = t1_seed - t2_seed
    X_test[ii, 0] = t1_seed
    X_test[ii, 1] = diff_seed
    X_test[ii, 2] = t1
    X_test[ii, 3] = year


### Scale data with appropriate scaler

In [ ]:
X_test = scaler.fit_transform(X_test)

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [ ]:
preds = base2.predict(X_test)

In [ ]:
clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()

Lastly, create your submission file!

In [ ]:
filename = 'KerasMLP_RankSeedsYear'
c=0
ext = '.csv'
if os.path.exists(filename+ext):
    while os.path.exists(filename+ext):
        c+=1
        filename = filename+'_'+str(c)
    df_sample_sub.to_csv(filename+ext, index=False)
else:
    df_sample_sub.to_csv(filename+ext, index=False)